In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import pandas as pd
import os
from keras.applications.vgg16 import VGG16
from keras.preprocessing import sequence
np.random.seed(7)
import cv2
from sklearn.model_selection import train_test_split

In [2]:
model = VGG16(weights='imagenet', include_top=False)

In [3]:
print(model.summary())

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [ ]:
XX = []
YY = []
j=0
base_path="../dataset/videos"
source_path=base_path
for child in os.listdir(source_path):
    print(child)
    sub_path = os.path.join(source_path, child)
    bsub_path = os.path.join(base_path, child)
    if os.path.isdir(sub_path):
        for data_file in os.listdir(sub_path):
            path=os.path.join(bsub_path,data_file)
            print(path)
            frames=[]
            cap=cv2.VideoCapture(path)
            fcount=cap.get(cv2.CAP_PROP_FRAME_COUNT)

            num_frames = 1000
            loop_limit = min(num_frames,fcount)
            print("Capturing {0} frames".format(num_frames))

            for i in range(0, loop_limit) :
                ret, frame = cap.read()
                frame = cv2.resize(frame,(224,224))
                frame = frame.reshape([-1,224,224,3])
                features_train=model.predict([frame])
                frames.append(features_train.flatten())

            last_frame= frames[-1]
            for i in range(loop_limit, num_frames):
                frames.append(last_frame)
            XX.append(frames)
            print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")

            cap.release()
            YY.append(child)
print(np.shape(XX))
print(np.shape(YY))

noVideos = np.shape(YY)[0]
print(noVideos)

0
../dataset/videos\0\uqnDGa1UN5o.mp4
Capturing 1000 frames


In [ ]:
X=np.array(XX)
print(X)
print(X.shape)

In [ ]:
print(YY)

In [ ]:
y=np.array(YY)
print(y.shape)

In [ ]:
print(XX)

In [ ]:
Y=y.reshape(-1,1)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
oh=OneHotEncoder()
y1=oh.fit_transform(Y)
lb=LabelBinarizer()
yy=lb.fit_transform(y1)
print(yy)

In [ ]:
modelClass = Sequential()
modelClass.add(LSTM(100, return_sequences=True, input_shape=(1000,25088)))
modelClass.add(LSTM(50, return_sequences=True, activation='relu'))
modelClass.add(LSTM(20, return_sequences=True, activation='relu'))
modelClass.add(LSTM(10, activation='relu'))
modelClass.add(Dense(noVideos, activation='softmax'))
modelClass.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelClass.summary())
modelClass.fit(X , yy , epochs=1000, batch_size=16)

In [ ]:
scores = modelClass.evaluate(X, yy, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))